In [3]:
!git clone https://github.com/traversaal-ai/AgentPro.git


Cloning into 'AgentPro'...
remote: Enumerating objects: 1423, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 1423 (delta 273), reused 182 (delta 182), pack-reused 1094 (from 2)
Receiving objects: 100% (1423/1423), 12.62 MiB | 30.78 MiB/s, done.
Resolving deltas: 100% (698/698), done.


In [1]:
# Install any needed libraries (some may already be installed)
!pip install openai requests scipy
!pip install dateparser

In [2]:
# Import libraries
import openai
import json
import requests
import numpy as np
from scipy.optimize import linprog
import os
from google.colab import userdata
from datetime import datetime, time
import dateparser
from datetime import datetime, timezone, timedelta

In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
print("Google API key loaded:", bool(GOOGLE_API_KEY))
print("OPEN AI API key loaded:", bool(OPENAI_API_KEY))

Google API key loaded: True
OPEN AI API key loaded: True


In [4]:
%cd AgentPro
!pip install -e .

/content/AgentPro
Obtaining file:///content/AgentPro
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for agentpro (pyproject.toml) ... done
  Created wheel for agentpro: filename=agentpro-0.1.0-0.editable-py3-none-any.whl size=7155 sha256=1a35ed8dcafc2debb82889d5b79eb57502016d4d66856ead1e568ddad29097f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-jkff2tqe/wheels/9c/c4/78/3256381eeab54c5a623877649478a807224b8b9955de374b97
Successfully built agentpro
  Attempting uninstall: agentpro
    Found existing installation: agentpro 0.1.0
    Uninstalling agentpro-0.1.0:
      Successfully uninstalled agentpro-0.1.0


In [5]:
from google.colab import userdata
import os

# Set keys from secrets
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

print("✅ API keys successfully loaded into environment.")


✅ API keys successfully loaded into environment.


In [6]:
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a catering expert."},
        {"role": "user", "content": "Create a meal plan for a tech conference in Dubai."}
    ]
)

print(response.choices[0].message.content)


Creating a meal plan for a tech conference in Dubai involves considering diverse dietary preferences, cultural norms, and the professional setting. Below is a suggested plan catering to various requirements over a three-day conference:

### Day 1

#### Breakfast
- **Continental Buffet:**
  - Fresh fruits: sliced watermelon, oranges, and berries
  - Assortment of pastries: croissants, muffins, and Danish pastries
  - Yogurts: plain and flavored options
  - Granola and muesli with milk (including soy and almond milk)

- **Beverages:**
  - Freshly brewed coffee and a selection of teas
  - Fresh orange juice and apple juice

#### Lunch
- **Starter:**
  - Lentil soup with coriander

- **Main Course:**
  - Grilled chicken breast with lemon and herbs
  - Roasted vegetable couscous (Vegetarian/Vegan option)
  - Baked Salmon fillet with dill sauce

- **Sides:**
  - Mixed greens salad with vinaigrette
  - Hummus and pita bread

- **Dessert:**
  - Baklava
  - Fresh fruit platter

- **Beverage:**


Building Agents

User Intent Agent

In [7]:
import sys
import os
import re
import json
import ast
import dateparser

# Add AgentPro to system path
sys.path.insert(0, "/content/AgentPro")

# ✅ AgentPro imports
from agentpro import ReactAgent, create_model
from agentpro.tools import UserInputTool  # optional

# ✅ Create model wrapper (this solves the .chat_completion error)
model = create_model(
    provider="openai",
    model_name="gpt-4o",
    api_key=os.environ["OPENAI_API_KEY"]
)

agent = ReactAgent(model=model, tools=[])

# 🧼 LLM response parser
def extract_json_from_response(response):
    cleaned = response.strip()
    cleaned = re.sub(r"```(?:json|python)?", "", cleaned)
    cleaned = re.sub(r"[^\x00-\x7F]+", "", cleaned).strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"(\{.*\}|\[.*\])", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON from LLM response:\n{cleaned}")

# 🚀 Main user intent extraction
def extract_user_intent():
    user_query = input("👤 Please describe the event you want to plan (type, location, guests, etc.):\n> ")

    prompt = (
      "You are an event planning assistant.\n"
      "Extract structured event info from the user query below.\n"
      "Return a **Python dictionary only**, no explanations or formatting.\n\n"
      "Required dictionary keys:\n"
      "- 'event_type': string\n"
      "- 'location': string\n"
      "- 'event_date': string (YYYY-MM-DD)\n"
      "- 'guest_count': integer\n"
      "- 'preferences': dictionary (include keys like 'stay_dates', 'guest_origins', etc.)\n"
      "- 'event_theme': string (optional)\n"
      "- 'meal_count': integer (optional)\n"
      "- 'transport_needs': string (optional)\n"
      "- 'sightseeing': boolean (optional)\n\n"
      "Missing values must be set to null.\n\n"
      "Output should be in pure JSON form and nothing else should be included which is not in JSON format"
      f"User input: {user_query}"
      "IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
)


    print("🔁 Querying GPT-4o via ReAct agent...")
    response = agent.run(prompt)
    raw_output = response.final_answer
    print("🧠 LLM RAW OUTPUT:\n", raw_output)

    intent_data = extract_json_from_response(raw_output)

    if intent_data.get("event_date"):
        parsed_date = dateparser.parse(intent_data["event_date"])
        intent_data["event_date"] = parsed_date.strftime("%Y-%m-%d") if parsed_date else None

    print("\n✅ Extracted User Intent:")
    print(json.dumps(intent_data, indent=2))
    return intent_data

# Run the extraction
extracted_intent = extract_user_intent()


👤 Please describe the event you want to plan (type, location, guests, etc.):
> Plan a wedding of 300 guests in islamabad on 13 June till 15 June 2025.
🔁 Querying GPT-4o via ReAct agent...
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: 



Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:
Thought: Your reasoning about action and observation.
Action: {"action_type": "<action_type>", "input": <input_data>}

Option 2 — When you're confident in the final response:
Thought: Now I know the answer that

Schedular Agent

In [9]:
import sys
import os
import json
import re

# Add AgentPro to path
sys.path.insert(0, "/content/AgentPro")

from agentpro import create_model, ReactAgent

# ✅ Create GPT-4o model client
model = create_model(
    provider="openai",
    model_name="gpt-4o",
    api_key=os.environ["OPENAI_API_KEY"]
)

agent = ReactAgent(model=model, tools=[])

# 📦 Extract structured list from LLM output
def extract_json_from_response(response):
    cleaned = response.strip()
    cleaned = re.sub(r"```(?:json|python)?", "", cleaned)
    cleaned = re.sub(r"[^\x00-\x7F]+", "", cleaned).strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"(\{.*\}|\[.*\])", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON from LLM response:\n{cleaned}")

# 📅 Scheduler Agent logic
def scheduler_agent(user_intent):
    """
    Uses GPT-4o to generate a high-level multi-day schedule based on intent.
    """
    event_type = user_intent.get("event_type")
    event_date = user_intent.get("event_date") or "a typical day"
    guest_count = user_intent.get("guest_count", "unknown")

    if not event_type:
        raise ValueError("User intent must include an event_type.")

    prompt = (
        f"You are an expert event planner. Create a high-level multi-day schedule for a {event_type} "
        f"beginning on {event_date} with around {guest_count} guests. Include major activities from arrival to departure:\n"
        f"airport pickups, hotel check-in, meals, sessions, sightseeing in free slots, and checkout.\n"
        f"Return the result as a Python list of dictionaries in this format:\n\n"
        f"Return only a valid JSON object with no code block markers, no explanation, no markdown, no labels — just the raw JSON."
        f"[{{\"day\": \"Day 1 - Arrival\", \"activities\": [\"Hotel check-in\", \"Welcome dinner\"]}}, ...]"
        f"IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
    )

    print("🧠 Prompting GPT-4o for schedule...")
    response = agent.run(prompt).final_answer
    schedule = extract_json_from_response(response)
    return schedule

# 🔁 Run in standalone mode
if __name__ == "__main__":
    try:
        user_intent = extracted_intent  # Make sure this is defined globally
    except NameError:
        raise Exception("❌ 'extracted_intent' not defined. Run User Intent Agent first.")

    print("\n🎯 Extracted User Intent:")
    print(json.dumps(user_intent, indent=2))

    print("\n📅 Generating high-level schedule...")
    schedule = scheduler_agent(user_intent)

    print("\n🗓️ Multi-day Itinerary:")
    for day in schedule:
        print(f"\n📌 {day['day']}")
        for act in day["activities"]:
            print(f"   - {act}")



🎯 Extracted User Intent:
{
  "event_type": "wedding",
  "location": "Islamabad",
  "event_date": "2025-06-13",
  "guest_count": 300,
  "preferences": {
    "stay_dates": "2025-06-13 to 2025-06-15",
    "guest_origins": null
  },
  "event_theme": null,
  "meal_count": null,
  "transport_needs": null,
  "sightseeing": null
}

📅 Generating high-level schedule...
🧠 Prompting GPT-4o for schedule...
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: 



Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:


Venue Agent

In [11]:
import os
import json
import requests
from agentpro import create_model, ReactAgent
from agentpro.tools import QuickInternetTool

# ✅ Load API keys from environment
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# ✅ Setup model + agent
model = create_model(provider="openai", model_name="gpt-4o", api_key=OPENAI_API_KEY)
agent = ReactAgent(model=model, tools=[QuickInternetTool()])  # tool not used yet, but available

# ✅ JSON extraction (robust for LLM output)
def extract_json_list(response):
    cleaned = response.strip()
    cleaned = cleaned.replace("```json", "").replace("```", "").strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"\[(.*?)\]", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(f"[{match.group(1)}]")
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON list from LLM response:\n{cleaned}")

# ✅ Get venue types for given event
def get_suitable_venue_types(event_type):
    prompt = (
        f"You are an expert event planner.\n"
        f"Suggest the 3 most suitable types of venues for a '{event_type}' event.\n"
        f"Return only a valid JSON object with no code block markers, no explanation, no markdown, no labels — just the raw JSON."
        f"Return only a valid JSON list like: [\"banquet hall\", \"outdoor garden\", \"conference center\"]"
        f"IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
    )
    response = agent.run(prompt)
    return extract_json_list(response.final_answer)

# ✅ Search Google Places for venue info
def search_venues(location, venue_type, limit=3):
    search_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{venue_type} in {location}",
        "key": GOOGLE_API_KEY
    }
    response = requests.get(search_url, params=params)
    results = response.json().get("results", [])[:limit]

    venues = []
    for result in results:
        place_id = result.get("place_id")
        details_url = "https://maps.googleapis.com/maps/api/place/details/json"
        details_params = {
            "place_id": place_id,
            "fields": "name,rating,formatted_address,formatted_phone_number,website",
            "key": GOOGLE_API_KEY
        }
        detail_resp = requests.get(details_url, params=details_params).json()
        detail = detail_resp.get("result", {})

        venues.append({
            "name": detail.get("name", "N/A"),
            "rating": detail.get("rating", "N/A"),
            "address": detail.get("formatted_address", "N/A"),
            "phone": detail.get("formatted_phone_number", "N/A"),
            "website": detail.get("website", "N/A")
        })

    return venues

# ✅ Main venue agent logic
def venue_agent(extracted_intent):
    event_type = extracted_intent.get("event_type")
    location = extracted_intent.get("location")

    if not event_type or not location:
        raise ValueError("❌ User intent must include both 'event_type' and 'location'.")

    print(f"\n🧠 Determining suitable venue types for: {event_type}")
    venue_types = get_suitable_venue_types(event_type)

    all_venues = []
    for vtype in venue_types:
        print(f"🔎 Searching: {vtype} in {location}")
        venues = search_venues(location, vtype)
        all_venues.append({
            "venue_type": vtype,
            "options": venues
        })

    return all_venues

# ✅ Run standalone
if __name__ == "__main__":
    print("\n🎯 Extracted User Intent:")
    print(json.dumps(extracted_intent, indent=2))

    venue_results = venue_agent(extracted_intent)

    print("\n📍 Recommended Venues:")
    for group in venue_results:
        print(f"\n🔹 Venue Type: {group['venue_type']}")
        for i, v in enumerate(group["options"], start=1):
            print(f"  {i}. {v['name']}")
            print(f"     ⭐ Rating: {v.get('rating', 'N/A')}")
            print(f"     📍 Address: {v.get('address', 'N/A')}")
            print(f"     ☎️ Phone: {v.get('phone', 'N/A')}")
            print(f"     🌐 Website: {v.get('website', 'N/A')}\n")



🎯 Extracted User Intent:
{
  "event_type": "wedding",
  "location": "Islamabad",
  "event_date": "2025-06-13",
  "guest_count": 300,
  "preferences": {
    "stay_dates": "2025-06-13 to 2025-06-15",
    "guest_origins": null
  },
  "event_theme": null,
  "meal_count": null,
  "transport_needs": null,
  "sightseeing": null
}

🧠 Determining suitable venue types for: wedding
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: search

Tool: Quick Internet Search
Description: Searches internet quickly using DuckDuckGo for a given query and returns top 5 results.
Action Type: search
Input Format: A search query as a string. Example: 'Latest advancements in AI'


Your task is to:
1. Think about what actio

Hotel Booking Agent

In [12]:
import os
import json
import re
import ast
import requests

from agentpro import create_model, ReactAgent

# ✅ Load environment keys
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]

# ✅ Agent setup
model = create_model(provider="openai", model_name="gpt-4o", api_key=OPENAI_API_KEY)
agent = ReactAgent(model=model, tools=[])

# ✅ JSON-safe parser
def extract_json_from_response(response):
    cleaned = response.strip()
    cleaned = re.sub(r"```(?:json|python)?", "", cleaned)
    cleaned = re.sub(r"[^\x00-\x7F]+", "", cleaned).strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"(\{.*\}|\[.*\])", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON from LLM response:\n{cleaned}")

# ✅ LLM: Analyze hotel needs based on intent + venues
def analyze_hotel_needs(user_intent, venues):
    prompt = (
        "You are a hotel planning assistant.\n"
        "Based on the event type, location, and venues, determine the hotel needs.\n"
        "Return a Python dictionary with:\n"
        "- 'hotel_type': e.g., 3-star, 5-star\n"
        "- 'room_requirements': {single: int, double: int, suite: int}\n"
        "- 'priorities': list of preferences (e.g., budget, proximity, Wi-Fi)\n"
        "- 'suggested_hotels': optional hotel suggestions with name, type, rooms, amenities\n\n"
        "Return only a valid JSON object with no code block markers, no explanation, no markdown, no labels — just the raw JSON."
        f"user_intent = {json.dumps(user_intent, indent=2)}\n"
        f"venues = {json.dumps(venues[:2], indent=2)}"
        "IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
    )
    response = agent.run(prompt)
    return extract_json_from_response(response.final_answer)

# ✅ Google API helpers
def get_place_details(place_id):
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "name,rating,formatted_address,formatted_phone_number,website",
        "key": GOOGLE_API_KEY
    }
    return requests.get(url, params=params).json().get("result", {})

def search_hotels_near_venue(venue_address, hotel_type, limit=3):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{hotel_type} hotel near {venue_address}",
        "key": GOOGLE_API_KEY
    }
    res = requests.get(url, params=params).json()
    results = res.get("results", [])[:limit]

    hotels = []
    for r in results:
        details = get_place_details(r.get("place_id", ""))
        hotels.append({
            "name": details.get("name", r.get("name")),
            "rating": details.get("rating", "N/A"),
            "address": details.get("formatted_address", r.get("formatted_address", "")),
            "phone": details.get("formatted_phone_number", "N/A"),
            "website": details.get("website", "N/A")
        })

    return hotels

# ✅ Main hotel booking agent
def hotel_booking_agent(user_intent, venue_data):
    all_venues = []
    for group in venue_data:
        all_venues.extend(group.get("options", []))

    # 🧠 LLM: Analyze hotel needs
    hotel_plan = analyze_hotel_needs(user_intent, all_venues)

    # ✅ Fix if hotel_plan is a list of dicts instead of a single dict
    if isinstance(hotel_plan, list) and len(hotel_plan) > 0:
      hotel_plan = hotel_plan[0]
    hotel_type = hotel_plan.get("hotel_type", "hotel")
    room_requirements = hotel_plan.get("room_requirements", {})
    priorities = hotel_plan.get("priorities", [])

    # 🔎 Search near each venue
    recommendations = {}
    for venue in all_venues:
        venue_name = venue.get("name")
        venue_address = venue.get("address") or venue.get("formatted_address")
        if not venue_name or not venue_address:
            continue

        hotels = search_hotels_near_venue(venue_address, hotel_type)
        recommendations[venue_name] = hotels

    return {
        "hotel_type": hotel_type,
        "room_requirements": room_requirements,
        "priorities": priorities,
        "recommendations": recommendations
    }

# ✅ Test entry point
if __name__ == "__main__":
    user_intent = extracted_intent
    venue_data = venue_agent(user_intent)

    print("\n🏨 Generating hotel suggestions...")
    hotel_output = hotel_booking_agent(user_intent, venue_data)

    print("\n🛏️ Room Requirements:")
    print(json.dumps(hotel_output["room_requirements"], indent=2))

    print("\n📌 Booking Priorities:")
    for p in hotel_output["priorities"]:
        print(f"- {p}")

    print("\n🏨 Hotel Recommendations by Venue:")
    for venue, hotels in hotel_output["recommendations"].items():
        print(f"\n📍 Near {venue}")
        for h in hotels:
            print(f"  - {h['name']} ({h['rating']})")
            print(f"     📍 {h['address']}")
            print(f"     ☎️ {h['phone']}")
            print(f"     🌐 {h['website']}")



🧠 Determining suitable venue types for: wedding
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: 



Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:
Thought: Your reasoning about action and observation.
Action: {"action_type": "<action_type>", "input": <input_data>}

Option 2 — When you're confident in the final response:
Thought: Now I know the answer that will be given in Final Answer.
Final Answer: Provide a complete, well-structured response that directly addresses the original question.



Vendor Agent

In [15]:
import os
import json
import requests
import ast
import re
from agentpro import ReactAgent, create_model
from agentpro.tools import QuickInternetTool

# ✅ Load keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Make sure it's set in environment
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")  # Make sure it's set in environment

# ✅ Create OpenAI model (GPT-4o or GPT-4)
model = create_model(
    provider="openai",
    model_name="gpt-4o",
    api_key=OPENAI_API_KEY
)

# ✅ Setup ReAct agent with tool
agent = ReactAgent(model=model, tools=[QuickInternetTool()])

# ✅ Utility to clean LLM list responses
def extract_json_list(response):
    cleaned = response.strip()
    cleaned = cleaned.replace("```json", "").replace("```", "").strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"\[(.*?)\]", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(f"[{match.group(1)}]")
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON list from LLM response:\n{cleaned}")

# ✅ LLM step to find vendor types
def get_required_vendor_types(event_type):
    prompt = (
        f"You are an expert event planner. What types of vendors are typically needed for a '{event_type}'? "
        "Return only a valid JSON object with no code block markers, no explanation, no markdown, no labels — just the raw JSON."
        "Return only a Python list like: [\"catering\", \"event lighting\", \"A/V equipment\"]"
        "IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
    )
    response = agent.run(prompt)
    return extract_json_list(response.final_answer)

# ✅ Google Places API
def search_vendors(location, vendor_type, limit=3):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{vendor_type} near {location}",
        "key": GOOGLE_API_KEY
    }
    res = requests.get(url, params=params).json()
    results = res.get("results", [])[:limit]

    vendors = []
    for result in results:
        place_id = result.get("place_id")
        details_url = "https://maps.googleapis.com/maps/api/place/details/json"
        details_params = {
            "place_id": place_id,
            "fields": "name,rating,formatted_address,formatted_phone_number,website",
            "key": GOOGLE_API_KEY
        }
        detail_resp = requests.get(details_url, params=details_params).json()
        d = detail_resp.get("result", {})
        vendors.append({
            "name": d.get("name"),
            "type": vendor_type,
            "rating": d.get("rating", "N/A"),
            "address": d.get("formatted_address", "N/A"),
            "phone": d.get("formatted_phone_number", "N/A"),
            "website": d.get("website", "N/A")
        })

    return vendors

# ✅ Main agent
def vendors_agent(user_intent):
    event_type = user_intent.get("event_type")
    location = user_intent.get("location")

    if not event_type or not location:
        raise ValueError("User intent must include both event_type and location.")

    vendor_types = get_required_vendor_types(event_type)
    all_vendors = []

    for vtype in vendor_types:
        print(f"🔎 Searching vendors for: {vtype}")
        vendors = search_vendors(location, vtype)
        all_vendors.append({
            "vendor_type": vtype,
            "options": vendors
        })

    return all_vendors

# ✅ Run if main
if __name__ == "__main__":

    print("\n🎯 Extracted Intent:")
    print(json.dumps(extracted_intent, indent=2))

    print("\n🔧 Finding vendors...")
    vendor_results = vendors_agent(extracted_intent)

    print("\n🛍️ Recommended Vendors:")
    for group in vendor_results:
        print(f"\n🔹 Vendor Type: {group['vendor_type']}")
        for idx, v in enumerate(group["options"], start=1):
            print(f"  {idx}. {v['name']} ({v['rating']})")
            print(f"     📍 {v['address']}")
            print(f"     ☎️ {v['phone']}")
            print(f"     🌐 {v['website']}")



🎯 Extracted Intent:
{
  "event_type": "wedding",
  "location": "Islamabad",
  "event_date": "2025-06-13",
  "guest_count": 300,
  "preferences": {
    "stay_dates": "2025-06-13 to 2025-06-15",
    "guest_origins": null
  },
  "event_theme": null,
  "meal_count": null,
  "transport_needs": null,
  "sightseeing": null
}

🔧 Finding vendors...
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: search

Tool: Quick Internet Search
Description: Searches internet quickly using DuckDuckGo for a given query and returns top 5 results.
Action Type: search
Input Format: A search query as a string. Example: 'Latest advancements in AI'


Your task is to:
1. Think about what action is required — Thought.
2. Take

Transport and Parking Agent

In [16]:
import os
import json
import ast
import re
import requests
from agentpro import ReactAgent, create_model

# ✅ Use OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "❌ Missing OPENAI_API_KEY in environment"

# ✅ Create model + ReAct agent with OpenAI GPT-4o
model = create_model(provider="openai", model_name="gpt-4o", api_key=OPENAI_API_KEY)
agent = ReactAgent(model=model)

def extract_json_from_response(response):
    cleaned = response.strip()
    cleaned = re.sub(r"```(?:json|python)?", "", cleaned)
    cleaned = re.sub(r"[^\x00-\x7F]+", "", cleaned).strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"(\{.*\}|\[.*\])", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON from LLM response:\n{cleaned}")

def analyze_transport_needs(user_intent, venues, schedule):
    prompt = (
        "You are a logistics planner. Based on this event, return:\n"
        "- A list of transport vendor types required\n"
        "- A dictionary of estimated vehicle needs by type\n\n"
        "Return only a valid JSON object with no code block markers, no explanation, no markdown, no labels — just the raw JSON."
        "Return this as a Python dictionary with two keys: 'vendor_types' and 'vehicle_estimates'.\n"
        "Example:\n"
        "{\n"
        "  \"vendor_types\": [\"shuttle service\", \"bus rental\"],\n"
        "  \"vehicle_estimates\": {\"cars\": 3, \"shuttles\": 2, \"buses\": 1}\n"
        "}\n\n"
        f"user_intent = {json.dumps(user_intent, indent=2)}\n"
        f"venues = {json.dumps(venues[:2], indent=2)}\n"
        f"schedule = {json.dumps(schedule[:2], indent=2)}"
        "IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
    )

    response = agent.run(prompt)
    cleaned = extract_json_from_response(response.final_answer)

    try:
        if isinstance(cleaned, dict):
            data = cleaned
        elif isinstance(cleaned, str):
            try:
                data = ast.literal_eval(cleaned)
            except Exception:
                data = json.loads(cleaned.replace("'", '"'))
        else:
            raise Exception("LLM returned unparseable content")

        if isinstance(data, dict) and "vendor_types" in data and "vehicle_estimates" in data:
            return data
        else:
            raise ValueError("Unexpected LLM output structure:\n" + json.dumps(data, indent=2))

    except Exception as e:
        raise Exception(f"LLM response could not be parsed:\n{cleaned}\n\nError: {e}")

def search_vendors(location, vendor_type, limit=3):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{vendor_type} near {location}",
        "key": os.getenv("GOOGLE_API_KEY")
    }
    res = requests.get(url, params=params).json()
    results = res.get("results", [])[:limit]

    vendors = []
    for r in results:
        vendors.append({
            "name": r.get("name"),
            "type": vendor_type,
            "address": r.get("formatted_address"),
            "rating": r.get("rating", "N/A")
        })
    return vendors

def get_nearest_transport_hubs(location):
    airport = search_vendors(location, "international airport", limit=1)
    railway = search_vendors(location, "intercity railway station", limit=1)
    bus = search_vendors(location, "intercity bus terminal", limit=1)

    return {
        "nearest_airport": airport[0] if airport else {"name": "N/A", "address": "N/A"},
        "nearest_railway_station": railway[0] if railway else {"name": "N/A", "address": "N/A"},
        "nearest_bus_station": bus[0] if bus else {"name": "N/A", "address": "N/A"},
    }

def find_parking_near_venues(venues):
    parking_results = {}
    for v in venues:
        venue_name = v.get("name")
        venue_address = v.get("address") or v.get("formatted_address")
        if not venue_name or not venue_address:
            continue

        url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
        params = {
            "query": f"parking near {venue_address}",
            "key": os.getenv("GOOGLE_API_KEY")
        }
        res = requests.get(url, params=params).json()
        results = res.get("results", [])[:3]

        parking_spots = [{
            "name": r.get("name"),
            "address": r.get("formatted_address"),
            "rating": r.get("rating", "N/A")
        } for r in results]

        parking_results[venue_name] = parking_spots
    return parking_results

def transport_parking_agent(user_intent, venue_data, schedule_data):
    location = user_intent.get("location")
    event_type = user_intent.get("event_type")
    if not location or not event_type:
        raise ValueError("Missing location or event_type.")

    all_venues = []
    for group in venue_data:
        all_venues.extend(group.get("options", []))

    transport_plan = analyze_transport_needs(user_intent, all_venues, schedule_data)
    vendor_types = transport_plan.get("vendor_types", [])
    vehicle_estimates = transport_plan.get("vehicle_estimates", {})

    vendor_recs = []
    for vtype in vendor_types:
        vendor_recs.append({
            "vendor_type": vtype,
            "options": search_vendors(location, vtype)
        })

    return {
        "transport_plan": {
            "vendor_types": vendor_types,
            "vehicle_estimates": vehicle_estimates
        },
        "vendor_recommendations": vendor_recs,
        "parking_near_venues": find_parking_near_venues(all_venues),
        "transport_hubs": get_nearest_transport_hubs(location)
    }

if __name__ == "__main__":
    user_intent = extracted_intent
    venue_data = venue_agent(user_intent)
    schedule_data = scheduler_agent(user_intent)
    transport_output = transport_parking_agent(user_intent, venue_data, schedule_data)

    print("\n🚗 Vehicle Estimates:")
    for k, v in transport_output["transport_plan"]["vehicle_estimates"].items():
        print(f"- {k.title()}: {v}")

    print("\n🛻 Transport Vendors:")
    for group in transport_output["vendor_recommendations"]:
        print(f"\n🔹 {group['vendor_type'].title()}")
        for v in group["options"]:
            print(f"  - {v['name']} ({v['rating']}) - {v['address']}")

    print("\n🅿️ Parking Near Venues:")
    for venue, lots in transport_output["parking_near_venues"].items():
        print(f"\n{venue}")
        for lot in lots:
            print(f"  - {lot['name']} ({lot['rating']}) - {lot['address']}")

    print("\n🚉 Nearest Transport Hubs:")
    hubs = transport_output.get("transport_hubs", {})
    print(f"🛫 Airport: {hubs.get('nearest_airport', {}).get('name', 'N/A')} - {hubs.get('nearest_airport', {}).get('address', 'N/A')}")
    print(f"🚆 Railway: {hubs.get('nearest_railway_station', {}).get('name', 'N/A')} - {hubs.get('nearest_railway_station', {}).get('address', 'N/A')}")
    print(f"🚌 Bus Station: {hubs.get('nearest_bus_station', {}).get('name', 'N/A')} - {hubs.get('nearest_bus_station', {}).get('address', 'N/A')}")



🧠 Determining suitable venue types for: wedding
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: 



Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:
Thought: Your reasoning about action and observation.
Action: {"action_type": "<action_type>", "input": <input_data>}

Option 2 — When you're confident in the final response:
Thought: Now I know the answer that will be given in Final Answer.
Final Answer: Provide a complete, well-structured response that directly addresses the original question.



Catering Agent

In [17]:
import sys
import os
import json
import requests

# AgentPro setup
sys.path.insert(0, "/content/AgentPro")
from agentpro import ReactAgent, create_model
from agentpro.tools import MealPlannerTool

# Load keys from environment
GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

# Create model and agent
model = create_model(provider="openai", model_name="gpt-4o", api_key=OPENAI_API_KEY)
meal_tool = MealPlannerTool()
agent = ReactAgent(model=model, tools=[meal_tool])
def extract_json_from_response(response):
    cleaned = response.strip()
    cleaned = re.sub(r"```(?:json|python)?", "", cleaned)
    cleaned = re.sub(r"[^\x00-\x7F]+", "", cleaned).strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"(\{.*\}|\[.*\])", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON from LLM response:\n{cleaned}")

# 🎯 Main catering agent logic
def catering_agent(user_intent, schedule):
    location = user_intent.get("location", "")
    if not location:
        raise ValueError("❌ Location missing in user intent.")

    # Run tool via ReAct agent
    llm_input = {
        "event_type": user_intent.get("event_type", "event"),
        "location": location,
        "schedule": schedule
    }

    print("🧠 Calling MealPlannerTool via ReAct agent...")
    response = agent.run(
        f"Use the meal planner tool to suggest an appropriate structured meal plan for the event in {location} based on the schedule."
        f" Use the schedule and intent below:\n\n{json.dumps(llm_input, indent=2)}"
        f"IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
    )

    if isinstance(response.final_answer, dict):
      meal_plan = response.final_answer.get("meal_plan", response.final_answer)
    else:
    # fallback for formatted string meal plan
      meal_plan = response.final_answer


    # 🔍 Use Google Places to find caterers
    caterer_recommendations = {}
    for meal in ["breakfast", "lunch", "dinner"]:  # Simplified; can use meal_plan keys
        vendors = search_caterers(location, meal)
        caterer_recommendations[meal] = vendors

    return {
        "catering_plan": meal_plan,
        "caterer_recommendations": caterer_recommendations
    }

# 📍 Google Places search
def search_caterers(location, meal_type, limit=3):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{meal_type} catering near {location}",
        "key": GOOGLE_API_KEY
    }
    res = requests.get(url, params=params).json()
    results = res.get("results", [])[:limit]

    caterers = []
    for r in results:
        place_id = r.get("place_id")
        detail_url = "https://maps.googleapis.com/maps/api/place/details/json"
        detail_params = {
            "place_id": place_id,
            "fields": "name,rating,formatted_address,formatted_phone_number,website",
            "key": GOOGLE_API_KEY
        }
        d = requests.get(detail_url, params=detail_params).json().get("result", {})
        caterers.append({
            "name": d.get("name"),
            "rating": d.get("rating", "N/A"),
            "address": d.get("formatted_address"),
            "phone": d.get("formatted_phone_number", "N/A"),
            "website": d.get("website", "N/A")
        })

    return caterers

# 🧪 Example usage
if __name__ == "__main__":
    user_intent = extracted_intent
    schedule = scheduler_agent(user_intent)

    print("\n🍽️ Running Catering Agent...")
    catering_result = catering_agent(user_intent, schedule)

    print("\n🧾 Catering Plan:")
    print(json.dumps(catering_result["catering_plan"], indent=2))

    print("\n👨‍🍳 Caterer Recommendations:")
    for meal, vendors in catering_result["caterer_recommendations"].items():
        print(f"\n🍴 {meal.title()}:")
        for v in vendors:
            print(f"  - {v['name']} ({v['rating']})")
            print(f"    📍 {v['address']}")
            print(f"    ☎️ {v['phone']}")
            print(f"    🌐 {v['website']}")


🧠 Prompting GPT-4o for schedule...
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: generate_meal_plan

Tool: Meal Planner Tool
Description: Generates a detailed meal plan for any event based on event type, location, and schedule.
Action Type: generate_meal_plan
Input Format: Dict with keys: 'event_type', 'location', and 'schedule'.


Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:
Thought: Your reasoning about action and observation.
Action: {"action_type": "<action_type>", "input": <input_dat

Sight Seeing Agent

In [18]:
import os
import json
import ast
import re
import requests
from agentpro import ReactAgent, create_model

# ✅ Use OpenAI API Key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "❌ Missing OPENAI_API_KEY in environment"

# ✅ Create ReAct Agent using GPT-4o
model = create_model(provider="openai", model_name="gpt-4o", api_key=OPENAI_API_KEY)
agent = ReactAgent(model=model, tools=[])

import re
import json

def extract_json_from_response(response):
    cleaned = response.strip()
    cleaned = re.sub(r"```(?:json|python)?", "", cleaned)
    cleaned = re.sub(r"[^\x00-\x7F]+", "", cleaned).strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"(\{.*\}|\[.*\])", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON from LLM response:\n{cleaned}")

def analyze_sightseeing_need(user_intent, schedule):
    if isinstance(schedule, dict):
        schedule = list(schedule.values())
    elif not isinstance(schedule, list):
        raise Exception("❌ Schedule must be a list or a dictionary of days.")

    sample_schedule = []
    for entry in schedule[:2]:
        if isinstance(entry, dict) and 'day' in entry and 'activities' in entry:
            activity = entry['activities'][0] if isinstance(entry['activities'], list) else str(entry['activities'])
            sample_schedule.append(f"{entry['day']}: {activity}")
    schedule_snippet = "\n".join(sample_schedule)

    prompt = (
        "You are a cultural planner helping design event schedules.\n"
        "Given the event type and a brief schedule summary, recommend whether sightseeing could be meaningfully integrated.\n"
        "If so, suggest the type and ideal time.\n\n"
        "Return only a valid JSON object with no code block markers, no explanation, no markdown, no labels — just the raw JSON."
        "Return ONLY a Python dictionary with the following keys:\n"
        "- 'is_required' (true/false)\n"
        "- 'sightseeing_type' (e.g. cultural, shopping, historical)\n"
        "- 'integration_notes' (e.g. best added on Day 2 afternoon)\n\n"
        "Do NOT return anything else. No text. No schedule. No explanation. and no emojis\n"
        "Example:\n"
        "{\n"
        "  \"is_required\": true,\n"
        "  \"sightseeing_type\": \"Cultural\",\n"
        "  \"integration_notes\": \"Schedule it on Day 2 afternoon.\"\n"
        "}\n\n"
        "IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
        f"Event type: {user_intent.get('event_type')}\n"
        f"Schedule summary:\n{schedule_snippet}"
    )

    response = agent.run(prompt)
    return extract_json_from_response(response.final_answer)

def get_sightseeing_places(location, theme="cultural", limit=5):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{theme} attractions near {location}",
        "key": os.getenv("GOOGLE_API_KEY")
    }
    res = requests.get(url, params=params).json()
    results = res.get("results", [])[:limit]

    return [{
        "name": r.get("name"),
        "rating": r.get("rating", "N/A"),
        "address": r.get("formatted_address"),
        "type": theme,
        "description": r.get("types", [])
    } for r in results]

def sightseeing_agent(user_intent, schedule_data):
    location = user_intent.get("location")
    if not location:
        raise ValueError("Location not specified in user intent.")

    analysis = analyze_sightseeing_need(user_intent, schedule_data)

    if not analysis.get("is_required", False):
        return {
            "is_required": False,
            "reason": analysis.get("reason", "LLM suggested sightseeing is not appropriate.")
        }

    theme = analysis.get("sightseeing_type", "cultural")
    places = get_sightseeing_places(location, theme)

    return {
        "is_required": True,
        "sightseeing_type": theme,
        "integration_notes": analysis.get("integration_notes", ""),
        "suggested_places": places
    }

if __name__ == "__main__":
    user_intent = extracted_intent
    schedule_data = scheduler_agent(user_intent)
    sightseeing = sightseeing_agent(user_intent, schedule_data)

    if not sightseeing["is_required"]:
        print("❌ Sightseeing not included:")
        print("Reason:", sightseeing.get("reason"))
    else:
        print("\n✅ Sightseeing Included!")
        print("Type:", sightseeing["sightseeing_type"])
        print("When:", sightseeing["integration_notes"])
        print("\n📍 Suggested Places:")
        for s in sightseeing["suggested_places"]:
            print(f"- {s['name']} ({s['rating']}) — {s['address']}")


🧠 Prompting GPT-4o for schedule...
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: 



Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:
Thought: Your reasoning about action and observation.
Action: {"action_type": "<action_type>", "input": <input_data>}

Option 2 — When you're confident in the final response:
Thought: Now I know the answer that will be given in Final Answer.
Final Answer: Provide a complete, well-structured response that directly addresses the original question.

### Important:

Event Theme Agent

In [19]:
import os
import json
import ast
import re
import requests
from agentpro import ReactAgent, create_model

# ✅ Use OpenAI API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "❌ Missing OPENAI_API_KEY in environment"

# ✅ Create ReAct Agent using GPT-4o
model = create_model(provider="openai", model_name="gpt-4o", api_key=OPENAI_API_KEY)
agent = ReactAgent(model=model, tools=[])


def extract_json_from_response(response):
    cleaned = response.strip()
    cleaned = re.sub(r"```(?:json|python)?", "", cleaned)
    cleaned = re.sub(r"[^\x00-\x7F]+", "", cleaned).strip()
    cleaned = cleaned.replace("“", '"').replace("”", '"').replace("‘", "'").replace("’", "'")

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        match = re.search(r"(\{.*\}|\[.*\])", cleaned, re.DOTALL)
        if match:
            try:
                return json.loads(match.group(0))
            except json.JSONDecodeError:
                pass
    raise ValueError(f"❌ Could not extract valid JSON from LLM response:\n{cleaned}")



def get_vendor_details(query, location, limit=3):
    url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    params = {
        "query": f"{query} near {location}",
        "key": os.getenv("GOOGLE_API_KEY")
    }
    res = requests.get(url, params=params).json()
    results = res.get("results", [])[:limit]

    vendors = []
    for r in results:
        place_id = r.get("place_id")
        details_url = "https://maps.googleapis.com/maps/api/place/details/json"
        details_params = {
            "place_id": place_id,
            "fields": "name,rating,formatted_address,formatted_phone_number,website",
            "key": os.getenv("GOOGLE_API_KEY")
        }
        details = requests.get(details_url, params=details_params).json().get("result", {})
        vendors.append({
            "name": details.get("name"),
            "address": details.get("formatted_address"),
            "phone": details.get("formatted_phone_number", "N/A"),
            "website": details.get("website", "N/A"),
            "rating": details.get("rating", "N/A"),
            "type": query
        })
    return vendors

def analyze_theme(user_intent):
    prompt = (
        "You are a creative event theme planner. Based on the event intent below, suggest a detailed event theme plan.\n"
        "Include:\n"
        "1. A creative theme name and style\n"
        "2. A list of decor elements needed (like penaflex, stage, lighting, florals)\n"
        "3. Any custom branding or stage setup suggestions\n"
        "Return only a valid JSON object with no code block markers, no explanation, no markdown, no labels — just the raw JSON."
        "Return as a JSON dictionary with keys: theme_name, style_description, required_elements, branding_notes.\n"
        "IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
        f"user_intent = {json.dumps(user_intent, indent=2)}"
    )
    response = agent.run(prompt)
    return extract_json_from_response(response.final_answer)

def theme_agent(user_intent):
    location = user_intent.get("location")
    if not location:
        raise ValueError("Location not found in user intent")

    theme_plan = analyze_theme(user_intent)

    required_elements = theme_plan.get("required_elements", [])
    if isinstance(required_elements, dict):
        required_elements = [item.get("item") for item in required_elements.get("decor", []) if item.get("item")]

    vendor_list = []
    for item in required_elements:
        vendor_list.extend(get_vendor_details(item, location))

    return {
        "recommended": {
            "theme_name": theme_plan.get("theme_name"),
            "style_description": theme_plan.get("style_description"),
            "branding_notes": theme_plan.get("branding_notes", ""),
            "required_elements": required_elements
        },
        "alternates": [],
        "vendors": vendor_list
    }

if __name__ == "__main__":
    user_intent = extracted_intent
    print("\n🎨 Generating Event Theme Plan...")
    theme_output = theme_agent(user_intent)

    recommended = theme_output["recommended"]

    print(f"\n🎭 Theme Name: {recommended['theme_name']}")
    print(f"🕋️ Description: {recommended['style_description']}")
    print(f"🧾 Branding Notes: {recommended['branding_notes']}")
    print(f"\n🛠️ Required Decor Elements:")
    for e in recommended['required_elements']:
        print(f"- {e}")

    print(f"\n🏢 Vendor Recommendations:")
    for v in theme_output['vendors']:
        print(f"\n🔹 {v['type'].title()} Vendor")
        print(f"  Name: {v['name']}")
        print(f"  📍 Address: {v['address']}")
        print(f"  ☎️ Phone: {v['phone']}")
        print(f"  🌐 Website: {v['website']}")
        print(f"  ⭐ Rating: {v['rating']}")



🎨 Generating Event Theme Plan...
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: 



Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:
Thought: Your reasoning about action and observation.
Action: {"action_type": "<action_type>", "input": <input_data>}

Option 2 — When you're confident in the final response:
Thought: Now I know the answer that will be given in Final Answer.
Final Answer: Provide a complete, well-structured response that directly addresses the original question.

### Important:


Weather Predictor

In [20]:
import requests
from datetime import datetime, timedelta
import os

# --- Weather Code Descriptions ---
WEATHER_CODE_DESCRIPTIONS = {
    0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
    45: "Fog", 48: "Depositing rime fog", 51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
    56: "Light freezing drizzle", 57: "Dense freezing drizzle", 61: "Slight rain", 63: "Moderate rain",
    65: "Heavy rain", 66: "Light freezing rain", 67: "Heavy freezing rain", 71: "Slight snow fall",
    73: "Moderate snow fall", 75: "Heavy snow fall", 77: "Snow grains", 80: "Slight rain showers",
    81: "Moderate rain showers", 82: "Violent rain showers", 85: "Slight snow showers",
    86: "Heavy snow showers", 95: "Thunderstorm", 96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail",
}

# --- Get Coordinates from Google Maps ---
def get_coordinates(city):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": city, "key": os.getenv("GOOGLE_API_KEY")}
    response = requests.get(url, params=params)
    data = response.json()
    if not data.get("results"):
        raise Exception(f"Could not determine coordinates for: {city}")
    location = data["results"][0]["geometry"]["location"]
    return location["lat"], location["lng"]

# --- Get Forecast from Open-Meteo ---
def get_open_meteo_forecast(lat, lon):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "daily": "temperature_2m_min,temperature_2m_max,weathercode",
        "timezone": "auto",
        "forecast_days": 16
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data.get("daily", {})

# --- Extract Event Dates from Schedule ---
def extract_event_dates(schedule, event_start_date):
    try:
        base_date = datetime.strptime(event_start_date, "%Y-%m-%d")
    except ValueError:
        raise Exception("Event start date must be in YYYY-MM-DD format.")

    return [(base_date + timedelta(days=i)).strftime("%Y-%m-%d") for i in range(len(schedule))]

# --- Filter Forecast for Event Dates ---
def filter_forecast(forecast_data, event_dates):
    filtered = []
    for date, t_min, t_max, code in zip(
        forecast_data.get("time", []),
        forecast_data.get("temperature_2m_min", []),
        forecast_data.get("temperature_2m_max", []),
        forecast_data.get("weathercode", [])
    ):
        if str(date).strip() in [d.strip() for d in event_dates]:
            filtered.append({
                "date": date,
                "min_temp": f"{t_min}°C",
                "max_temp": f"{t_max}°C",
                "description": WEATHER_CODE_DESCRIPTIONS.get(code, "Unknown")
            })
    return filtered

# --- Main Weather Agent ---
def weather_predictor_agent(user_intent: dict, schedule: list) -> list:
    city = user_intent.get("location")
    event_start_date = user_intent.get("event_date")

    if not city or not event_start_date:
        raise Exception("User intent must include both 'location' and 'event_date'.")

    lat, lon = get_coordinates(city)
    forecast_data = get_open_meteo_forecast(lat, lon)
    event_dates = extract_event_dates(schedule, event_start_date)
    return filter_forecast(forecast_data, event_dates)

# --- Test Block ---
if __name__ == "__main__":
    try:
        user_intent = extracted_intent
        print("\n🌤️ Fetching weather forecast (Open-Meteo)...")
        forecast = weather_predictor_agent(user_intent, schedule)

        print("\n📆 Weather Forecast for Event Days:")
        if not forecast:
            print("No forecast available.")
        else:
            for day in forecast:
                print(f"{day['date']}: 🌤️ {day['description']}")
                print(f"    🌡️ Temp: {day['min_temp']} to {day['max_temp']}")

    except Exception as e:
        print(f"❌ Error in Weather Agent: {e}")



🌤️ Fetching weather forecast (Open-Meteo)...

📆 Weather Forecast for Event Days:
2025-06-13: 🌤️ Light drizzle
    🌡️ Temp: 35.5°C to 46.6°C
2025-06-14: 🌤️ Light drizzle
    🌡️ Temp: 34.1°C to 45.8°C
2025-06-15: 🌤️ Mainly clear
    🌡️ Temp: 31.3°C to 44.3°C
2025-06-16: 🌤️ Mainly clear
    🌡️ Temp: 33.6°C to 45.6°C


Orchestrator Agent

In [21]:
import json
from agentpro import ReactAgent, create_model
import os
#from user_intent_agent import extract_user_intent

# ✅ OpenAI model setup
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "❌ Missing OPENAI_API_KEY in environment"

model = create_model(provider="openai", model_name="gpt-4o", api_key=OPENAI_API_KEY)
agent = ReactAgent(model=model, tools=[])

import json
import ast
def summarize_output(section_name, data):
    prompt = (
        f"You are a summarizer for an event planner agent.\n"
        f"Please summarize the key information from the '{section_name}' section in bullet points, focusing on top recommended options only.\n\n"
        f"{section_name} data:\n{json.dumps(data, indent=2)}\n\n"
        "IMPORTANT: Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. "
        "Do NOT output thoughts, actions, or any reasoning."
    )

    print(f"\n📦 Summarizing section: {section_name}")
    response = agent.run(prompt)

    if not hasattr(response, "final_answer") or not response.final_answer.strip().startswith("{"):
        raise ValueError(f"❌ LLM did not return usable JSON. Raw output:\n{response.final_answer}")

    return extract_json_from_response(response.final_answer)

def summarize_output(section_name, data):
    prompt = (
        f"You are a summarizer for an event planner agent.\n"
        f"Return only a valid JSON object with no code block markers, no explanation, no markdown, no labels — just the raw JSON."
        f"Please summarize the key information from the '{section_name}' section in bullet points, focusing on top recommended options only.\n\n"
        f"{section_name} data:\n{json.dumps(data, indent=2)}"
        "IMPORTANT: In each sub agent, Respond immediately with the final JSON object using only `Final Answer:` followed by the valid JSON. Do NOT output thoughts, actions, or any reasoning."
    )
    return agent.run(prompt).final_answer

def orchestrated_event_plan(user_intent):
    print("\n🚀 Launching Orchestrator Agent...")

    # 1. Extract User Intent
    #from your_user_intent_module import extract_user_intent  # adjust import if needed
    #user_intent = extract_user_intent(user_query)

    print("\n🎯 User Intent:")
    print(json.dumps(user_intent, indent=2))

    # 2. Venue Options
    venue_results = venue_agent(user_intent)
    venue_summary = summarize_output("Venues", venue_results)

    # 3. Vendor Recommendations
    vendor_results = vendors_agent(user_intent)
    vendor_summary = summarize_output("Vendors", vendor_results)

    # 4. Event Schedule
    schedule = scheduler_agent(user_intent)

    # 5. Transport & Parking
    transport_output = transport_parking_agent(user_intent, venue_results, schedule)
    transport_summary = summarize_output("Transportation", transport_output)

    # 6. Hotel Options
    hotel_output = hotel_booking_agent(user_intent, venue_results)
    hotel_summary = summarize_output("Hotels", hotel_output)

    # 7. Sightseeing Plan
    sightseeing = sightseeing_agent(user_intent, schedule)
    sightseeing_summary = summarize_output("Sightseeing", sightseeing)

    # 8. Catering Options
    catering_result = catering_agent(user_intent, schedule)
    catering_summary = summarize_output("Catering", catering_result)

    # 9. Theme Plan
    theme_output = theme_agent(user_intent)
    theme_summary = summarize_output("Theme", theme_output)

    # 10. Weather Forecast
    forecast = weather_predictor_agent(user_intent, schedule)

    # 🧠 Final Output Structure
    event_plan = {
        "intent": user_intent,
        "summary": {
            "venues": venue_summary,
            "vendors": vendor_summary,
            "transportation": transport_summary,
            "hotels": hotel_summary,
            "sightseeing": sightseeing_summary,
            "catering": catering_summary,
            "theme": theme_summary,
        },
        "details": {
            "venues": venue_results,
            "vendors": vendor_results,
            "schedule": schedule,
            "transportation": transport_output,
            "hotels": hotel_output,
            "sightseeing": sightseeing,
            "catering": catering_result,
            "theme": theme_output,
            "weather_forecast": forecast,
        }
    }

    print("\n✅ Event Planning Complete!")
    return event_plan

# ✅ Run it
if __name__ == "__main__":
    extracted_intent = extract_user_intent()
    user_query = extracted_intent
    plan = orchestrated_event_plan(extracted_intent)

    print("\n📋 Summary Output:")
    print(json.dumps(plan["summary"], indent=2))


👤 Please describe the event you want to plan (type, location, guests, etc.):
> A wedding in Islamabad on 13 June 2025 for 300 guests. guests will stay from 12 till 14 June 2025
🔁 Querying GPT-4o via ReAct agent...
================================================== Iteration 1 
✅  [Debug] Sending System Prompt (with history) to LLM:
You are an AI assistant that follows the ReAct (Reasoning + Acting) pattern.
        
Your goal is to help users by breaking down complex tasks into a series of thought-out steps and actions.

You have access to these tools: 



Your task is to:
1. Think about what action is required — Thought.
2. Take an appropriate action — Action.
3. Repeat Thought/Action as needed until you find the final answer.

### Format (Choose only one per response)

Option 1 — When action is needed:
Thought: Your reasoning about action and observation.
Action: {"action_type": "<action_type>", "input": <input_data>}

Option 2 — When you're confident in the final response:
Thought: 